In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
from cicada import transcript

handler = logging.StreamHandler()
handler.setFormatter(transcript.Formatter())
transcript.logger.addHandler(handler)

In [3]:
import numpy

from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator
from cicada.encoding import FixedPoint

def main(communicator):
    transcript.log("Let's share some secrets!", comm=communicator)
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!", comm=communicator)
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!", comm=communicator)
    c = protocol.reveal(c_share)

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3);

Player 0: Let's share some secrets!
Player 1: Let's share some secrets!
Player 2: Let's share some secrets!
Player 0: --> 1 PRZS 2107302276397721018
Player 1: --> 2 PRZS 7148852727544971610
Player 2: --> 0 PRZS 5008483476944089773
Player 1: <-- 0 PRZS 2107302276397721018
Player 0: <-- 2 PRZS 5008483476944089773
Player 2: <-- 1 PRZS 7148852727544971610
Player 1: AdditiveProtocolSuite.share [2] 47
Player 0: FixedPoint.encode [2] 8
Player 2: AdditiveProtocolSuite.share [2] 55
Player 1: FixedPoint.encode [3] 12
Player 0: Field.inplace_add [33, 8] 33
Player 2: AdditiveProtocolSuite.share [3] 125
Player 1: Field.inplace_add [12, 12] 12
Player 0: AdditiveProtocolSuite.share [2] 33
Player 2: Let's add some secrets!
Player 1: AdditiveProtocolSuite.share [3] 12
Player 0: AdditiveProtocolSuite.share [3] 2
Player 1: Let's add some secrets!
Player 2: Field.add [55, 125] 53
Player 0: Let's add some secrets!
Player 1: Field.add [47, 12] 59
Player 2: Let's reveal the results!
Player 0: Field.add [33, 